In [1]:
#!pip install -U scikit-learn
#!pip install gensim
#!pip install dagshub

  Using cached Cython-0.29.28-py2.py3-none-any.whl (983 kB)
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.33
    Uninstalling Cython-0.29.33:
      Successfully uninstalled Cython-0.29.33


In [2]:
import sys
!{sys.executable} -m pip install scikit-learn

You should consider upgrading via the 'C:\Users\15143\anaconda3\envs\MLops310523\python.exe -m pip install --upgrade pip' command.


In [31]:
import numpy as np 
import docx2txt
import pickle
import re, os
import string
import pandas as pd
import seaborn as sns
import nltk
import warnings
import docx2txt
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from wordcloud import WordCloud
from wordcloud import WordCloud ,STOPWORDS
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
import gensim.downloader as api
from gensim.utils import simple_preprocess
from sklearn.svm import SVC
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
import scipy.sparse
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [32]:
#Data
df = pd.read_csv('Resume.csv')
# create list of all categories
categories = np.sort(df['Category'].unique())
# lenght
df['length_str'] = df['Resume_str'].str.len()

In [33]:
#Feature Engineering
#Cleaning
def clean_text(text):
       
    text = text.lower() # lowercase text
    text = text.replace('\d+', '') # remove digits
    text = re.compile('[/(){}\[\]\|@,;]').sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = re.compile('[^0-9a-z #+_]').sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in set(stopwords.words('english'))) # remove stopwors from text
    
    
    text = re.sub('http\S+\s*', ' ', text)  # remove URLs
    text = re.sub('RT|cc', ' ', text)  # remove RT and cc
    text = re.sub('#\S+', '', text)  # remove hashtags
    text = re.sub('@\S+', '  ', text)  # remove mentions
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', text)  # remove punctuations
    text = re.sub(r'[^\x00-\x7f]',r' ', text) #remplacer tous les caractères non ASCII
    text = re.sub('\s+', ' ', text)  # remove extra whitespace
    
    # remove non-english characters, punctuation and numbers
    text = re.sub('[^a-zA-Z]', ' ', text) 
    # tokenize word
    text = nltk.tokenize.word_tokenize(text) 
    # remove stop words
    text = [w for w in text if not w in nltk.corpus.stopwords.words('english')]
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(text) for text in text]

    return ' '.join(text)

def filter_text(text): 
    text = ' '.join(word for word in text.split() if word not in set(stopwords.words('english')).union(['state','summary', 'city','company', 'name', 'skill'])) # remove stopwors from text
    return text

#Encode the labels into numeric

def set_code(row):
    if row["Category"] == "ACCOUNTANT": return 1
    elif row["Category"] == "ADVOCATE": return 2
    elif row["Category"] == "AGRICULTURE": return 3
    elif row["Category"] == "APPAREL": return 4
    elif row["Category"] == "ARTS": return 5
    elif row["Category"] == "AUTOMOBILE": return 6
    elif row["Category"] == "AVIATION": return 7
    elif row["Category"] == "BANKING": return 8
    elif row["Category"] == "BPO": return 9
    elif row["Category"] == "BUSINESS-DEVELOPMENT": return 10
    elif row["Category"] == "CHEF": return 11
    elif row["Category"] == "CONSTRUCTION": return 12
    elif row["Category"] == "CONSULTANT": return 13
    elif row["Category"] == "DESIGNER": return 14
    elif row["Category"] == "DIGITAL-MEDIA": return 15
    elif row["Category"] == "ENGINEERING": return 16
    elif row["Category"] == "FINANCE": return 17
    elif row["Category"] == "HEALTHCARE": return 18
    elif row["Category"] == "HR": return 19
    elif row["Category"] == "INFORMATION-TECHNOLOGY": return 20
    elif row["Category"] == "PUBLIC-RELATIONS": return 21
    elif row["Category"] == "SALES": return 22
    elif row["Category"] == "TEACHER": return 23
    elif row["Category"] == "FITNESS": return 24
    else: return 25  

In [34]:
#cleaning
df = df.assign(target=df.apply(set_code, axis=1))
df['Resume_Clean'] = df['Resume_str'].apply(clean_text)
df['length_Clean'] = df['Resume_Clean'].str.len()
df['Resume_filtered'] = df['Resume_Clean'].apply(lambda w: filter_text(w))
df['length_filtered'] = df['Resume_filtered'].str.len()

In [55]:
# Créer un nouveau DataFrame df1 à partir de df
df1 = df.copy()

In [57]:
# Chemin vers le dossier contenant votre code
#folder_path = r"C:\Users\15143\Desktop\MLops_Syn_Project"

# Nom du fichier de sauvegarde
#file_name = "df1.csv"

# Chemin complet du fichier de sauvegarde
#save_path = folder_path + "/" + file_name

# Enregistrement du DataFrame en tant que fichier CSV
#df1.to_csv(save_path, index=False)


In [ ]:
#df1 = pd.read_csv('df1.csv')

In [36]:
# Supprimer toutes les colonnes sauf 'Colonne1' et 'Colonne2'
colonnes_a_garder = ['Resume_filtered', 'target']
df1 = df1[colonnes_a_garder]

In [37]:
#Balance
def upsample_classes(data, target):
    
    lst = list(data[target].unique())
    
    classes = []
    for c in lst:
        classes.append(data[data[target]==c])
    
    length = 0
    class_lab = None
    for c in classes:
        if len(c)>length:
            length=len(c)
            class_lab = c
    class_lab = class_lab[target].unique()[0]
    
    regroup = pd.concat(classes)
    maj_class = regroup[regroup[target]==class_lab]

    lst.remove(class_lab)
    
    new_classes=[]
    for i in lst:
        new_classes.append(resample(data[data[target]==i],replace=True, n_samples=len(maj_class)))

    minority_classes = pd.concat(new_classes)
    upsample = pd.concat([regroup[regroup[target]==class_lab],minority_classes])

    return upsample

In [38]:
df_balanced = (upsample_classes(df1,'target'))

In [39]:
#split
#!pip install --upgrade scikit-learn
X_train, X_test, y_train, y_test = train_test_split(df_balanced['Resume_filtered'], df_balanced['target'],stratify=df_balanced['target'], test_size=0.2, random_state=8)

In [40]:
# Bag Of Words
# vectorize text data
vectorizer = CountVectorizer()
conuntvectorizer_train = vectorizer.fit_transform(X_train).astype(float)
conuntvectorizer_test = vectorizer.transform(X_test).astype(float)

In [ ]:
#MLFlow work Starts from here

In [ ]:
#mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts --host 0.0.0.0 --port 5000

In [ ]:
#Function to create an experiment in MLFlow and log parameters, metrics and artifacts files like images etc.

In [41]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = OneVsRestClassifier(svm.SVC(C=4.0, kernel='linear', degree=3, gamma='auto'))
SVM.fit(conuntvectorizer_train, y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(conuntvectorizer_test)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

SVM Accuracy Score ->  87.5


In [59]:
#import os
#import mlflow
#import mlflow.sklearn
#from sklearn.metrics import accuracy_score


#mlflow.set_tracking_uri("https://dagshub.com/herras.adil/MLops_Syn_Project_0906.mlflow")

#os.environ["MLFLOW_TRACKING_USERNAME"] = "herras.adil"
#os.environ["MLFLOW_TRACKING_PASSWORD"] = "e7d3465f7a9ab4f2d60b2f026a876a494be74f26"

In [64]:

mlflow.set_tracking_uri("http://localhost:5000") #uncomment this line if you want to use any database like sqlite as backend storage for model
#mlflow.set_tracking_uri("https://dagshub.com/herras.adil/MLops_Syn_Project_0906.mlflow")

os.environ["MLFLOW_TRACKING_USERNAME"] = "herras.adil"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "e7d3465f7a9ab4f2d60b2f026a876a494be74f26"

experiment_name = "SVM_classifier"
mlflow.set_experiment(experiment_name)
run_name="SVM_2"

with mlflow.start_run():
    
    # Enregistrer le nom de l'expérience
    mlflow.set_tag("mlflow.experimentName", experiment_name)
    mlflow.set_tag("mlflow.run_name", run_name)

        
    # Enregistrer les paramètres de l'exécution
    run_params = {"C": 4.0, "kernel":'linear', "degree":3, "gamma":'auto'}
    mlflow.log_params(run_params)
        

    # Définir les noms des métriques pour l'exécution
    #run_metrics = ["accuracy"]
    # Calculer et enregistrer les métriques de l'exécution
    accuracy=accuracy_score(predictions_SVM, y_test)*100
    mlflow.log_metrics({"accuracy": accuracy})
        
    mlflow.sklearn.log_model(SVM, "SVM_classifier")
        
    mlflow.set_tag("tag1", "SVM_classifier")
    mlflow.set_tags({"tag2":"", "tag3":""})
            
print('Run - %s is logged to Experiment - %s' %(run_name, experiment_name))

Run - SVM_2 is logged to Experiment - SVM_classifier


In [104]:
# Random Forest
Random_Forest  = OneVsRestClassifier(RandomForestClassifier(random_state=1,max_depth= 80, min_samples_leaf= 1, min_samples_split= 5, n_estimators= 4000))
Random_Forest.fit(conuntvectorizer_train, y_train)
model_predictions = Random_Forest.predict(conuntvectorizer_test)
print('Random Forest Accuracy: ', accuracy_score(y_test, model_predictions)*100)

Random Forest Accuracy:  84.72222222222221


In [105]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5000") #uncomment this line if you want to use any database like sqlite as backend storage for model
experiment_name = "Random_Forest_classifier"
mlflow.set_experiment(experiment_name)

run_name="Random_Forest_classifier_2"
    
with mlflow.start_run():
        
       # Enregistrer les paramètres de l'exécution
    run_params = {"random_state": 1, "min_samples_leaf":1, "max_depth":80, "min_samples_split":5, "n_estimators":4000}
    mlflow.log_params(run_params)
        

        # Définir les noms des métriques pour l'exécution
    #run_metrics = ["accuracy"]
        # Calculer et enregistrer les métriques de l'exécution
    accuracy=accuracy_score(y_test, model_predictions)*100
    mlflow.log_metrics({"accuracy": accuracy})
        
    mlflow.sklearn.log_model(Random_Forest, "Random_Forest_classifier")
        
    mlflow.set_tag("tag1", "Random_Forest")
    mlflow.set_tags({"tag2":"", "tag3":""})
            
print('Run - %s is logged to Experiment - %s' %(run_name, experiment_name))

Run - Random_Forest_classifier_2 is logged to Experiment - Random_Forest_classifier


In [110]:
# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
gbc = GradientBoostingClassifier()
ovr_classifier  = OneVsRestClassifier(gbc)
ovr_classifier.fit(conuntvectorizer_train, y_train)
model_predictions = ovr_classifier .predict(conuntvectorizer_test)
print('GradientBoostingClassifier Accuracy: ', accuracy_score(y_test, model_predictions)*100)

GradientBoostingClassifier Accuracy:  88.19444444444444


In [100]:
# Get the training parameters of the classifier
params = GradientBoostingClassifier.get_params()

# Print the training parameters
for param, value in params.items():
    print(f"{param}: {value}")

estimator__ccp_alpha: 0.0
estimator__criterion: friedman_mse
estimator__init: None
estimator__learning_rate: 0.1
estimator__loss: log_loss
estimator__max_depth: 3
estimator__max_features: None
estimator__max_leaf_nodes: None
estimator__min_impurity_decrease: 0.0
estimator__min_samples_leaf: 1
estimator__min_samples_split: 2
estimator__min_weight_fraction_leaf: 0.0
estimator__n_estimators: 100
estimator__n_iter_no_change: None
estimator__random_state: None
estimator__subsample: 1.0
estimator__tol: 0.0001
estimator__validation_fraction: 0.1
estimator__verbose: 0
estimator__warm_start: False
estimator: GradientBoostingClassifier()
n_jobs: None
verbose: 0


In [111]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5000") #uncomment this line if you want to use any database like sqlite as backend storage for model
experiment_name = "GradientBoostingClassifier_classifier"
mlflow.set_experiment(experiment_name)

run_name="GradientBoostingClassifier_classifier_2"
    
with mlflow.start_run():
        
       # Enregistrer les paramètres de l'exécution
    run_params = {"max_depth": 3, "learning_rate": 0.1, "n_estimators":100}
    mlflow.log_params(run_params)
        

        # Définir les noms des métriques pour l'exécution
    #run_metrics = ["accuracy"]
        # Calculer et enregistrer les métriques de l'exécution
    accuracy=accuracy_score(y_test, model_predictions)*100
    mlflow.log_metrics({"accuracy": accuracy})
        
    mlflow.sklearn.log_model(GradientBoostingClassifier, "GradientBoostingClassifier_classifier")
        
    mlflow.set_tag("tag1", "GradientBoostingClassifier_classifier")
    mlflow.set_tags({"tag2":"", "tag3":""})
            
print('Run - %s is logged to Experiment - %s' %(run_name, experiment_name))

2023/06/06 18:25:47 INFO mlflow.tracking.fluent: Experiment with name 'GradientBoostingClassifier_classifier' does not exist. Creating a new experiment.


Run - GradientBoostingClassifier_classifier_2 is logged to Experiment - GradientBoostingClassifier_classifier


In [113]:
## GradientBoostingClassifier
# liste des modèles que vous souhaitez entraîner
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

models = [
    
    {
        'name': 'GradientBoostingClassifier',
        'model': OneVsRestClassifier(GradientBoostingClassifier()),
        'params': {
            'estimator__learning_rate': [0.05, 0.1],
            'estimator__n_estimators': [100, 150],
            'estimator__max_depth': [2, 3, 4]
        }
    }
]

# Parcourez la liste des modèles et entraînez-les avec différents hyperparamètres
for model in models:
    
    print(f"Training {model['name']}...")
    model_tuning = GridSearchCV(model['model'], model['params'], cv=3)
    model_tuning.fit(conuntvectorizer_train,y_train)
    print(f"Best parameters for {model['name']}: {model_tuning.best_params_}")
    
# Calcul de l'accuracy score sur l'ensemble d'entraînement
accuracy = model_tuning.score(conuntvectorizer_train,y_train)

print("GradientBoostingClassifier Accuracy Score on Training Set -> ", accuracy * 100)

# Calcul de l'accuracy score sur l'ensemble de test
accuracy_test = model_tuning.score(conuntvectorizer_test, y_test)
print(f"Accuracy Score on Test Set for {model['name']}: {accuracy_test * 100}")

Training GradientBoostingClassifier...
Best parameters for GradientBoostingClassifier: {'estimator__learning_rate': 0.1, 'estimator__max_depth': 2, 'estimator__n_estimators': 100}
GradientBoostingClassifier Accuracy Score on Training Set ->  99.73958333333334
Accuracy Score on Test Set for GradientBoostingClassifier: 86.80555555555556


In [114]:
# Diviser les données en ensembles d'entraînement, de validation et de test
X_train_1, X_val, y_train_1, y_val = train_test_split(conuntvectorizer_train, y_train, test_size=0.2, random_state=42)

# Initialiser le modèle
gb = OneVsRestClassifier(GradientBoostingClassifier())

# Spécifier les paramètres à tester avec GridSearchCV
params = {
    'estimator__learning_rate': [0.1],
    'estimator__max_depth': [2],
    'estimator__n_estimators': [100]
}

# Initialiser GridSearchCV
clf = GridSearchCV(gb, params)


# Entraîner le modèle avec les meilleurs paramètres sur l'ensemble d'entraînement
clf.fit(X_train_1, y_train_1)

# Évaluer les performances sur l'ensemble de validation
accuracy_val = clf.score(X_val, y_val)*100
print("Accuracy Score on Validation Set:", accuracy_val)

# Une fois que vous êtes satisfait des performances sur l'ensemble de validation,
# vous pouvez utiliser l'ensemble d'entraînement complet pour entraîner le modèle final

# Entraîner le modèle final avec l'ensemble d'entraînement complet
clf.fit(conuntvectorizer_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = clf.predict(conuntvectorizer_test)

# Calculer l'accuracy score
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy Score on Test Set -> ", accuracy * 100)

Accuracy Score on Validation Set: 88.93709327548807
Accuracy Score on Test Set ->  87.67361111111111


In [116]:
import mlflow
from datetime import datetime
mlflow.set_tracking_uri("http://localhost:5000") #uncomment this line if you want to use any database like sqlite as backend storage for model
experiment_name = "GradientBoostingClassifier_classifier_Optimized_" + str(datetime.now().strftime("%d-%m-%y"))
mlflow.set_experiment(experiment_name) 

run_name="GradientBoostingClassifier_classifier_1_Optimized_" + str(datetime.now().strftime("%d-%m-%y"))
    
with mlflow.start_run():
        
       # Enregistrer les paramètres de l'exécution
    run_params = {"max_depth": 2, "learning_rate": 0.1, "n_estimators":100}
    mlflow.log_params(run_params)
        

        # Définir les noms des métriques pour l'exécution
    #run_metrics = ["accuracy"]
        # Calculer et enregistrer les métriques de l'exécution
    accuracy=accuracy_score(y_test, y_pred)*100
    mlflow.log_metrics({"accuracy": accuracy})
        
    mlflow.sklearn.log_model(clf, "GradientBoostingClassifier_classifier_Optimized")
        
    mlflow.set_tag("tag1", "GradientBoostingClassifier_classifier_Optimized")
    mlflow.set_tags({"tag2":"", "tag3":""})
            
print('Run - %s is logged to Experiment - %s' %(run_name, experiment_name))

Run - GradientBoostingClassifier_classifier_1_Optimized_07-06-23 is logged to Experiment - GradientBoostingClassifier_classifier_Optimized_07-06-23


In [138]:
#Copie from Mlflow : Predict Data
import mlflow
logged_model = 'runs:/546fef3f478541ac91750bbe39908caa/GradientBoostingClassifier_classifier_Optimized'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict
loaded_model.predict(conuntvectorizer_test)

In [139]:
# Décodage des prédictions
# Map the predicted sentiment to the real value
reverse_mapping = { 1: "ACCOUNTANT", 2: "ADVOCATE", 3: "AGRICULTURE", 4: "APPAREL",
    5: "ARTS", 6: "AUTOMOBILE", 7: "AVIATION", 8: "BANKING",
    9: "BPO", 10: "BUSINESS-DEVELOPMENT", 11: "CHEF", 12: "CONSTRUCTION",
    13: "CONSULTANT", 14: "DESIGNER", 15: "DIGITAL-MEDIA", 16: "ENGINEERING",
    17: "FINANCE", 18: "HEALTHCARE", 19: "HR", 20: "INFORMATION-TECHNOLOGY",
    21: "PUBLIC-RELATIONS", 22: "SALES", 23: "TEACHER", 24: "FITNESS", 25: "UNKNOWN"}

# Make the prediction
Resume_Categorie  = loaded_model.predict(conuntvectorizer_test)
# Map the predicted sentiment to the category name
categories = [reverse_mapping[num] for num in Resume_Categorie]
print("Predicted Category:", categories)

In [ ]:
#Adding an MLflow Model to the Model Registry

In [125]:
#Method 2
import mlflow
with mlflow.start_run():
    result = mlflow.register_model(
        'runs:/546fef3f478541ac91750bbe39908caa/GradientBoostingClassifier_classifier_Optimized',
        "GradientBoostingClassifier_Optimized"
    )

Successfully registered model 'GradientBoostingClassifier_Optimized'.
2023/06/07 08:53:28 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: GradientBoostingClassifier_Optimized, version 1
Created version '1' of model 'GradientBoostingClassifier_Optimized'.


In [ ]:
#Fetching an MLflow Model from the Model Registry
#Fetch a specific model version

In [126]:
#Transitioning an MLflow Model’s Stage
client = mlflow.tracking.MlflowClient()
client.transition_model_version_stage(
    name="GradientBoostingClassifier_Optimized",
    version=1,
    stage="Production"
)

<ModelVersion: creation_timestamp=1686142408843, current_stage='Production', description='', last_updated_timestamp=1686142483649, name='GradientBoostingClassifier_Optimized', run_id='546fef3f478541ac91750bbe39908caa', run_link='', source='./artifacts/5/546fef3f478541ac91750bbe39908caa/artifacts/GradientBoostingClassifier_classifier_Optimized', status='READY', status_message='', tags={}, user_id='', version='1'>

In [135]:
# Make the prediction
import mlflow.pyfunc
model_name = "GradientBoostingClassifier_Optimized"
version=1
model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{version}")

y_pred = model.predict(conuntvectorizer_test)

In [140]:
# Make the prediction
Resume_Categorie  = y_pred
# Map the predicted sentiment to the category name
categories = [reverse_mapping[num] for num in Resume_Categorie]
print("Predicted Category:", categories)

In [ ]:
# Prédiction sur de nouvelles données
#new_data = docx2txt.process('python-sample-resume.docx')
new_data = df.iloc[28].Resume_str

# Preprocess the text
# Preprocess the text
new_data = clean_text(new_data)
new_data = filter_text(new_data)

# Create a new CountVectorizer with the same vocabulary as the original vectorizer
new_vectorizer = CountVectorizer(vocabulary=vectorizer.vocabulary_)

# Vectorize the preprocessed text using the new vectorizer
features = new_vectorizer.transform([new_data])

# Ensure the number of features matches the expected number
if features.shape[1] != 33645:
    # Pad the features with zeros to match the expected number of features
    padding = scipy.sparse.csr_matrix((features.shape[0], 33645 - features.shape[1]))
    features = scipy.sparse.hstack([features, padding])

# Make the prediction
sentiment = clf.predict(features)[0]

print("Predicted Sentiment:", sentiment)

# Décodage des prédictions
# Map the predicted sentiment to the real value
reverse_mapping = { 1: "ACCOUNTANT", 2: "ADVOCATE", 3: "AGRICULTURE", 4: "APPAREL",
    5: "ARTS", 6: "AUTOMOBILE", 7: "AVIATION", 8: "BANKING",
    9: "BPO", 10: "BUSINESS-DEVELOPMENT", 11: "CHEF", 12: "CONSTRUCTION",
    13: "CONSULTANT", 14: "DESIGNER", 15: "DIGITAL-MEDIA", 16: "ENGINEERING",
    17: "FINANCE", 18: "HEALTHCARE", 19: "HR", 20: "INFORMATION-TECHNOLOGY",
    21: "PUBLIC-RELATIONS", 22: "SALES", 23: "TEACHER", 24: "FITNESS", 25: "UNKNOWN"}

# Make the prediction
sentiment = clf.predict(features)[0]
# Map the predicted sentiment to the category name
predicted_category = reverse_mapping.get(sentiment, "UNKNOWN")
print("Predicted Category:", predicted_category)

In [6]:
#!python --version

Python 3.9.7


In [1]:
#!pip freeze > requirements.txt

Le processus ne peut pas accéder au fichier car ce fichier est utilisé par un autre processus.
